In [ ]:
# Market Making strategy

"Parameters"
execution_fees = 0.01 # 1% of the half quoted spread
Q_max = 20
inventory = 10
wealth = 10000 #in EUR
traded_quantity = 2

T = 25182
sigma = 0.3
gamma = 2
k = 0.3
alpha = (k / 2) * gamma * sigma**2
A = 0.9
eta = A * (1 + (gamma / k))**(-(1 + (k / gamma)))

def function_vector(alpha, Q_max, eta, T, t):
    N = 2 * Q_max + 1
    inv = np.array([-q for q in range(1, Q_max+1)][::-1] + [q for q in range(Q_max+1)])
    a = [alpha * q**2 for q in inv]
    L = np.diag(a)
    M = L + np.eye(N,N,1) * (-eta) + np.eye(N,N,-1) * (-eta)

    v = np.dot(expm((-1) * M * (1 - t / T)), np.ones(N).T)
    v_ = {}
    for i in range(len(inv)):
        v_[inv[i]] = v[i]
    return v_

def bid_quote(t, T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price, is_mid_price = True, approx = True, time_asymp = False):
    if not approx:
        if inventory != Q_max:
            v = function_vector(alpha, Q_max, eta, T, t)
            s_ = mid_price - (1 / k) * log(v[inventory] / v[inventory + 1]) - (1 / gamma) * log(1 + gamma / k)
            return s_
        else:
            print("inventory limit error")
    elif time_asymp:
        s_ = mid_price - (1 / gamma) * log(1 + (gamma / k)) - ((1 + 2 * inventory) / 2) * sqrt((sigma**2 * gamma / (2 * k * A)) * (1 + (gamma / k))**(1+ (k / gamma)))
        return s_
    elif approx:
        if is_mid_price:
            s_ = mid_price - (1 / gamma) * log(1 + (gamma / k)) - ((1 + 2 * inventory) / 2) * gamma * sigma**2 * (1 - t / T)
        else:
            reservation_price = mid_price - inventory * gamma * sigma**2 * (1 - t / T)
            s_ = reservation_price - (1 / gamma) * log(1 + (gamma / k)) - ((1 + 2 * inventory) / 2) * gamma * sigma**2 * (1 - t / T)
        return s_
        
def ask_quote(t, T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price, is_mid_price = True, approx = True, time_asymp = False):
    if not approx:
        if inventory != -Q_max:
            v = function_vector(alpha, Q_max, eta, T, t)
            s_ = mid_price + (1 / k) * log(v[inventory] / v[inventory - 1]) + (1 / gamma) * log(1 + gamma / k)
            return s_
        else:
            print("inventory limit error")   
    elif time_asymp:
        s_ = mid_price + (1 / gamma) * log(1 + (gamma / k)) + ((1 - 2 * inventory) / 2) * sqrt((sigma**2 * gamma / (2 * k * A)) * (1 + (gamma / k))**(1+ (k / gamma)))
        return s_
    elif approx:
        if is_mid_price:
            s_ = mid_price + (1 / gamma) * log(1 + (gamma / k)) + ((1 - 2 * inventory) / 2) * gamma * sigma**2 * (1 - t / T)
        else:
            reservation_price = mid_price - inventory * gamma * sigma**2 * (1 - t / T)
            s_ = reservation_price + (1 / gamma) * log(1 + (gamma / k)) + ((1 - 2 * inventory) / 2) * gamma * sigma**2 * (1 - t / T)
        return s_

def dynamic_strategy(T, alpha, eta, gamma, sigma, k, Q_max, inventory):
    #Trades DataFrame adjustment:
    
    """ Upper/lower bound for the strategy timeframe. """
    prices = trades[trades["TIME"][0] <= trades["TIME"]][trades["TIME"] <= trades["TIME"][0] + T]
    
    initial_mid_price = (prices["ASK0"][0] + prices["BID0"][0]) / 2
    PnL = [(0, 0)]
    inv_historic = [(inventory, 0)]
    optimal_quotes = []
    reservation = []
    
    tick_size = 0.01 # To be adapted 
    
    delta_time = 3   # Waiting time in seconds, if our limit orders are not executed in 3 seconds after they are posted, we cancel them and re-quote again using the new LOB state.
    
    approxi = True
    approxi_asymp = False
    is_mid_price = False
    
    
    mid_price_before = (prices["ASK0"][0] + prices["BID0"][0]) / 2
        
    optimal_ask_quote = ask_quote(prices["TIME"][0] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_before, is_mid_price, approxi, approxi_asymp)
    optimal_bid_quote = bid_quote(prices["TIME"][0] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_before, is_mid_price, approxi, approxi_asymp)
        
    previous_quotes = (optimal_bid_quote, optimal_ask_quote, 0)
    optimal_quotes.append((optimal_bid_quote, optimal_ask_quote, 0))
    
    for i in range(len(prices["PRICE"])):
        mid_price = (prices["ASK0"][i] + prices["BID0"][i]) / 2
        reservation.append((mid_price - inventory * gamma * sigma**2 * (1 - (prices["TIME"][i] - prices["TIME"][0]) / T), prices["TIME"][i] - prices["TIME"][0]))
        
        if prices["TIME"][i] - previous_quotes[2] <= delta_time:
            price_execution = prices["PRICE"][i]
            optimal_ask_quote, optimal_bid_quote = previous_quotes[1], previous_quotes[0]

            if prices["ORDER_SIDE"][i] == "B":
                # Round the optimal quote to the nearest tick
                optimal_ask_quote = np.round(optimal_ask_quote, 2)

                if optimal_ask_quote <= price_execution or price_execution == 0:
                    # Inventory update
                    inventory -= traded_quantity
                    inv_historic.append((inventory, prices["TIME"][i] - prices["TIME"][0]))

                    # P&L update
                    PnL.append((PnL[-1][0] + (optimal_ask_quote - initial_mid_price - abs(optimal_ask_quote - mid_price) * execution_fees) * traded_quantity, prices["TIME"][i] - prices["TIME"][0]))
                    print(optimal_ask_quote, price_execution, "B")
                    ##
                    mid_price_after = (prices["Next_ASK0"][i] + prices["Next_BID0"][i]) / 2
                    optimal_ask_quote = ask_quote(prices["TIME"][i] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_after, is_mid_price, approxi, approxi_asymp)
                    optimal_bid_quote = bid_quote(prices["TIME"][i] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_after, is_mid_price, approxi, approxi_asymp)
                    
                    previous_quotes = (optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0])
                    optimal_quotes.append((optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0]))

                else:
                    PnL.append((PnL[-1][0], prices["TIME"][i] - prices["TIME"][0]))
                    optimal_quotes.append((previous_quotes[0], previous_quotes[1], prices["TIME"][i] - prices["TIME"][0]))
            else:
                # Round the optimal quote to the nearest tick
                optimal_bid_quote = np.round(optimal_bid_quote, 2)

                if optimal_bid_quote >= price_execution or price_execution == 0:
                    # Inventory update
                    inventory += traded_quantity
                    inv_historic.append((inventory, prices["TIME"][i] - prices["TIME"][0]))

                    #P&L update
                    PnL.append((PnL[-1][0] + (initial_mid_price - optimal_bid_quote - abs(optimal_bid_quote - mid_price) * execution_fees) * traded_quantity, prices["TIME"][i] - prices["TIME"][0]))
                    print(optimal_bid_quote, price_execution, "A")
                    ##
                    mid_price_after = (prices["Next_ASK0"][i] + prices["Next_BID0"][i]) / 2
                    optimal_ask_quote = ask_quote(prices["TIME"][i] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_after, is_mid_price, approxi, approxi_asymp)
                    optimal_bid_quote = bid_quote(prices["TIME"][i] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_after, is_mid_price, approxi, approxi_asymp)

                    previous_quotes = (optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0])
                    optimal_quotes.append((optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0]))
                else:
                    PnL.append((PnL[-1][0], prices["TIME"][i] - prices["TIME"][0]))
                    optimal_quotes.append((previous_quotes[0], previous_quotes[1], prices["TIME"][i] - prices["TIME"][0]))
        else:
            # Quotes Update
            mid_price_before = (prices["ASK0"][i] + prices["BID0"][i]) / 2
        
            optimal_ask_quote = ask_quote(prices["TIME"][i] - prices["TIME"][i], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_before, is_mid_price, approxi, approxi_asymp)
            optimal_bid_quote = bid_quote(prices["TIME"][i] - prices["TIME"][i], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_before, is_mid_price, approxi, approxi_asymp)

            previous_quotes = (optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0])
            optimal_quotes.append((optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0]))
    
            price_execution = prices["PRICE"][i]
        
            if prices["ORDER_SIDE"][i] == "B":
                # Round the optimal quote to the nearest tick
                optimal_ask_quote = np.round(optimal_ask_quote, 2)

                if optimal_ask_quote <= price_execution or price_execution == 0:
                    # Inventory update
                    inventory -= traded_quantity
                    inv_historic.append((inventory, prices["TIME"][i] - prices["TIME"][0]))

                    #P&L update
                    PnL.append((PnL[-1][0] + (optimal_ask_quote - initial_mid_price - abs(optimal_ask_quote - mid_price) * execution_fees) * traded_quantity, prices["TIME"][i] - prices["TIME"][0]))
                    print(optimal_ask_quote, price_execution, "B")
                    ##
                    mid_price_after = (prices["Next_ASK0"][i] + prices["Next_BID0"][i]) / 2
                    optimal_ask_quote = ask_quote(prices["TIME"][i] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_after, is_mid_price, approxi, approxi_asymp)
                    optimal_bid_quote = bid_quote(prices["TIME"][i] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_after, is_mid_price, approxi, approxi_asymp)

                    previous_quotes = (optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0])
                    optimal_quotes.append((optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0]))
                else:
                    PnL.append((PnL[-1][0], prices["TIME"][i] - prices["TIME"][0]))
                    optimal_quotes.append((previous_quotes[0], previous_quotes[1], prices["TIME"][i] - prices["TIME"][0]))

            else:
                # Round the optimal quote to the nearest tick
                optimal_bid_quote = np.round(optimal_bid_quote, 2)

                if optimal_bid_quote >= price_execution:
                    # Inventory update
                    inventory += traded_quantity
                    inv_historic.append((inventory, prices["TIME"][i] - prices["TIME"][0]))
                    
                    #P&L update
                    PnL.append((PnL[-1][0] + (initial_mid_price - optimal_bid_quote - abs(optimal_bid_quote - mid_price) * execution_fees) * traded_quantity, prices["TIME"][i] - prices["TIME"][0]))
                    print(optimal_bid_quote, price_execution, "A")
                    ##
                    mid_price_after = (prices["Next_ASK0"][i] + prices["Next_BID0"][i]) / 2
                    optimal_ask_quote = ask_quote(prices["TIME"][i] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_after, is_mid_price, approxi, approxi_asymp)
                    optimal_bid_quote = bid_quote(prices["TIME"][i] - prices["TIME"][0], T, alpha, eta, gamma, sigma, k, Q_max, inventory, mid_price_after, is_mid_price, approxi, approxi_asymp)

                    previous_quotes = (optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0])
                    optimal_quotes.append((optimal_bid_quote, optimal_ask_quote, prices["TIME"][i] - prices["TIME"][0]))
                else:
                    PnL.append((PnL[-1][0], prices["TIME"][i] - prices["TIME"][0]))
                    optimal_quotes.append((previous_quotes[0], previous_quotes[1], prices["TIME"][i] - prices["TIME"][0]))


    return [PnL, inv_historic, optimal_quotes, reservation]